## Firma Digital
En esta notebook se presenta un demo de uso de hmac, key derivation function y de firmas digitales.

### 1. HMAC
HMAC provee: Integridad del mensaje, Autenticación (quien conoce la clave)<br>
Utiliza: Una función hash (SHA-256 en este caso), Una clave secreta compartida<br>
No provee: No repudio, Firma pública (eso se cubre luego con RSA)

In [1]:
import hmac
import hashlib
import os

# Ruta al archivo (multiplataforma)
archivo = os.path.join("archivos", "2prueba10.txt")

# Lectura del contenido del archivo como bytes
with open(archivo, "rb") as f:
    mensaje = f.read()
print("Mensaje: ",mensaje.decode("utf-8").rstrip("\n"))

# Clave secreta compartida
clave_secreta = b"clave_super_secreta"
print("Clave secreta: ", clave_secreta.decode("utf-8"))


Mensaje:  unaprueba
Clave secreta:  clave_super_secreta


In [2]:
# Generación del HMAC usando SHA-256
hmac_generado = hmac.new(
    key=clave_secreta,
    msg=mensaje,
    digestmod=hashlib.sha256
).hexdigest()

print("Archivo:", archivo)
print("Contenido del archivo:")
print(mensaje.decode(errors="replace"))
print("\nHMAC (SHA-256):", hmac_generado)


Archivo: archivos/2prueba10.txt
Contenido del archivo:
unaprueba


HMAC (SHA-256): f9c33f35695590da39567e0e9790c20d7fc4ac9314729d87c053aaada7f7c7e7


In [3]:
# Verificación del HMAC
hmac_recibido = hmac_generado

# Recalcular el HMAC a partir del archivo
with open(archivo, "rb") as f:
    mensaje_verificacion = f.read()

hmac_calculado = hmac.new(
    key=clave_secreta,
    msg=mensaje_verificacion,
    digestmod=hashlib.sha256
).hexdigest()

# Comparación segura
if hmac.compare_digest(hmac_calculado, hmac_recibido):
    print("✔ El HMAC es válido: el archivo no fue modificado")
else:
    print("✘ El HMAC NO es válido: el archivo fue alterado")


✔ El HMAC es válido: el archivo no fue modificado


### 2. Key Derivation Function
A continuación se presenta un esquema de Key Derivation Function (KDF) usando HKDF (RFC 5869), implementado con la librería estándar de Python (hashlib + hmac), sin dependencias externas.

Contexto del demo:<br>
* Input Key Material (IKM): contenido del archivo archivos/2prueba10.txt<br>
* Función hash: SHA-256<br>
* Resultado: clave derivada de longitud controlada (por ejemplo, 32 bytes)

In [4]:
import hmac
import hashlib
import os

# Ruta al archivo: se usa el mismo del ejemplo anterior
# Lectura del archivo como bytes (IKM=b'unaprueba')
ikm = mensaje

# Parámetros HKDF
salt = b"salt_demo_hkdf"          # Puede ser público
info = b"demo-hkdf-sha256"        # Contexto / propósito
longitud_clave = 32               # Bytes (256 bits)

In [5]:
# --- HKDF-Extract ---
prk = hmac.new(
    key=salt,
    msg=ikm,
    digestmod=hashlib.sha256
).digest()

# --- HKDF-Expand ---
okm = b""
bloque_anterior = b""
contador = 1

while len(okm) < longitud_clave:
    bloque = hmac.new(
        key=prk,
        msg=bloque_anterior + info + bytes([contador]),
        digestmod=hashlib.sha256
    ).digest()
    okm += bloque
    bloque_anterior = bloque
    contador += 1

clave_derivada = okm[:longitud_clave]

# Presentación
print("IKM (archivo):", ikm.decode(errors="replace").rstrip("\n"))
print("Salt:", salt.decode())
print("Info:", info.decode())
print("Clave derivada (hex):", clave_derivada.hex())


IKM (archivo): unaprueba
Salt: salt_demo_hkdf
Info: demo-hkdf-sha256
Clave derivada (hex): c3957487a447a33bc762a771efa5591126b8ee8416730dbad08fdcfbc0bf55a7


La clave deivada generada puede usarse inmediatamente para:<br>
* hmac
* AES
* Clave de sesion

### 3. Firma Digital
Se implementará firma digital con RSA y SHA256. El mensaje que se firmará es el contenido del archivo.

In [6]:
#%pip install pycryptodome

In [7]:
# imports
from Crypto.PublicKey import RSA
import os
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256


In [8]:
# Key Generation:
# Rutas de claves
private_key_path = os.path.join("keys", "rsa_private_key.pem")
public_key_path = os.path.join("keys", "rsa_public_key.pem")

# Crear carpeta keys si no existe
os.makedirs("keys", exist_ok=True)

# Verificar si las claves ya existen
if os.path.exists(private_key_path) and os.path.exists(public_key_path):
    print("Claves RSA existentes encontradas. No se generan claves nuevas.")
else:
    # Generar par de claves RSA (2048 bits)
    key = RSA.generate(2048)

    private_key = key.export_key()
    public_key = key.publickey().export_key()

    # Guardar clave privada
    with open(private_key_path, "wb") as f:
        f.write(private_key)

    # Guardar clave pública
    with open(public_key_path, "wb") as f:
        f.write(public_key)

    print("Claves RSA generadas y almacenadas en la carpeta 'keys'")

Claves RSA generadas y almacenadas en la carpeta 'keys'


In [11]:
# Firma Digital del mensaje usando la clave privada RSA + SHA-256
# Cargar clave privada:
with open(os.path.join("keys", "rsa_private_key.pem"), "rb") as f:
    private_key = RSA.import_key(f.read())

# Calcular hash del mensaje
hash_mensaje = SHA256.new(mensaje)
print("Hash del mensaje (SHA-256):", hash_mensaje.hexdigest())
print("firma digital: ",)

# Firmar
firma = pkcs1_15.new(private_key).sign(hash_mensaje)

print("Mensaje:", mensaje.decode(errors="replace").rstrip("\n"))
print("Firma digital (hex):", pkcs1_15.new(private_key).sign(hash_mensaje).hex())


Hash del mensaje (SHA-256): ec1cdb03d513a7e8ee7ede003dc99263dc7f7fd8a6286ebe8f77db9d187c5bb5
firma digital: 
Mensaje: unaprueba
Firma digital (hex): 21654e016e46658df4bf5787e82ffc5cba6e2d73e0eadb66901d4cfa16462576bed6f6d47d73035d6cb74c3097b6c982a5627308b027ce5ad4ee4fa5dbd5caafa5c80d065453b6dc7de6fa6134e14bf5accaa91b249cc304c478eb96a85558a25400cd4caa565858ba63d39fef181ed0addcb654f1355725c0a5d468244daba1ff94af48be6833add86a7a49c6776ad9d5067a5f5f754e46be70723211dffacbda9ba33a63f48dd0a029236c10c4cde52752c98796952d6ce42b0ec21094586679050498ab8175c107dbb397755a868fdeaf2e8e0f60b2951dfeb4769393ad9565ba611cb2b0f455f4037808cb61f1d7a6735cd2f1f1ebde2cb7eeba1e28215b


In [ ]:
# Verificación de la firma Digital
# Cargar clave pública
with open(os.path.join("keys", "rsa_public_key.pem"), "rb") as f:
    public_key = RSA.import_key(f.read())

# Verificar la firma
"""
Para verificar la firma hecha con la clave privada, se utiliza la clave pública 
para desencriptar la firma y obtener el hash original.
Luego: 
Se compara este hash con el hash del mensaje, calculado en el paso anterior"""
try:
    pkcs1_15.new(public_key).verify(hash_mensaje, firma)
    print("✔ Firma válida: autenticidad e integridad verificadas")
except (ValueError, TypeError):
    print("✘ Firma inválida: el mensaje fue alterado o la clave no corresponde")

✔ Firma válida: autenticidad e integridad verificadas


#### Nota importante: <br>
La firma no protege directamente el mensaje, Protege el hash del mensaje

### Diagrama de Flujo comparativo

In [ ]:
#                    ┌──────────────────────────────┐
#                    │  Archivo de entrada          │
#                    │  archivos/2prueba10.txt      │
#                    └───────────────┬──────────────┘
#                                    │
#                                    ▼
#                           Lectura en bytes
#                                    │
#        ┌───────────────────────────┼───────────────────────────┐
#        │                           │                           │
#        ▼                           ▼                           ▼
#   SHA-256                     HMAC-SHA256                  HKDF-SHA256
# (hash puro)              (clave simétrica)             (derivación de claves)
#        │                           │                           │
#        │                           │                           │
#        ▼                           ▼                           ▼
# hash_mensaje            Código de autenticación        Clave(s) derivada(s)
#        │                   (integridad + auth)        (material criptográfico)
#        │
#        │
#        ▼
# RSA (clave privada)
#        │
#        ▼
# Firma digital
#        │
#        ▼
# ┌─────────────────────────────────────────────────────────────┐
# │        Contenido firmado + firma digital                    │
# └─────────────────────────────────────────────────────────────┘
#                                    │
#                                    ▼
#                        Proceso de verificación
#                                    │
#                                    ▼
#                    RSA (clave pública) + SHA-256
#                                    │
#                                    ▼
#                      Verificación de integridad,
#                    autenticidad y no repudio


### Conclusiones
Este demo muestra, de forma progresiva y coherente, los tres pilares criptográficos más utilizados en sistemas modernos:

* HMAC: Permite garantizar la integridad y autenticidad del contenido cuando existe una clave secreta compartida. Es eficiente y adecuado para sistemas cerrados o canales autenticados.

* HKDF: Resuelve el problema de cómo obtener claves criptográficamente seguras a partir de material inicial, evitando reutilización insegura de claves y separando dominios criptográficos.

* Firma digital RSA: Introduce el modelo asimétrico, donde la verificación no requiere secretos compartidos y se obtiene no repudio, característica esencial en sistemas legales, auditorías y comunicaciones abiertas.

El uso conjunto de estos mecanismos refleja prácticas reales en protocolos ampliamente adoptados (TLS, PKI, sistemas de firma, APIs seguras), y permite comprender claramente:
* qué se protege (el hash),
* cómo se protege (HMAC o firma),
* y con qué tipo de confianza (simétrica o asimétrica).

Desde el punto de vista didáctico, el enfoque basado en un único archivo como mensaje permite visualizar de manera directa cómo cualquier modificación del contenido rompe la cadena de confianza, validando los principios fundamentales de la criptografía aplicada.